# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
# Import files
from multi_ddpg import Agent
import torch
from collections import OrderedDict, namedtuple, deque
import matplotlib.pyplot as plt

In [6]:
# Instance of the ddpg agent
agent = Agent(state_size, action_size, random_seed=2)

### Training loop
def run_ddpg_multi(n_episodes=5000, queue=100):        
    """Deep Deterministic Policy Gradient learning for Reacher Unity Environment.
    
    Params Input
    ==========
        n_episode (int): maximum number of episodes
        queue (int): number of consecutive episodes 
        
    Params Output
    ==========
        scores_all (list of floats): are the scores collected at the end of each episode
        
    """
    
    ##Inizialization
    scores_window = deque(maxlen=queue)     
    scores_all = []                         
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      
        states = env_info.vector_observations
        agent.reset()
        scores = np.zeros(2)
        
        ## Training loop of each episode
        while True:
            actions = agent.act(states, add_noise=True)
            env_info = env.step(actions)[brain_name]           
            next_states = env_info.vector_observations         
            rewards = env_info.rewards                         
            dones = env_info.local_done
            agent.step(states, actions, rewards, next_states, dones)
            scores += rewards                                  
            states = next_states                               
            if np.any(dones):                                  
                break

        scores_window.append(np.max(scores))
        scores_all.append(np.max(scores))
        
        print('Episode {} ## Reward 1 agent: {:.3f} ##  Reward 2 : {:.3f} ## Max: {:.3f}'.format(
                i_episode, scores[0], scores[1], np.max(scores)))
        print('Average Score of last 100 episodes: {:.3f}'.format(np.mean(scores_window) ))

        if np.mean(scores_window)>=0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.3f}'.format(
                i_episode, np.mean(scores_window)))
            torch.save(agent.actor_local0.state_dict(), 'checkpoint_actor0.pth')
            torch.save(agent.critic_local0.state_dict(), 'checkpoint_critic0.pth')
            torch.save(agent.actor_local1.state_dict(), 'checkpoint_actor1.pth')
            torch.save(agent.critic_local1.state_dict(), 'checkpoint_critic1.pth')
            break

    return scores_all

In [ ]:
scores = run_ddpg_multi()

average_scores = []
temp_deque = deque(maxlen=100)
for s in scores:
    temp_deque.append(s)
    average_scores.append(np.mean(temp_deque))

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(range(1, len(scores)+1), scores)
ax.plot(range(1, len(average_scores)+1), average_scores, c='y', label='mean over 100 episodes')
ax.set_xlabel('Episode #', fontsize=14)
ax.set_ylabel('Score', fontsize=14)
plt.legend()
plt.show()

Episode 1 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 2 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 3 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 4 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 5 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 6 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 7 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 8 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 9 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100

Episode 73 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 74 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 75 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 76 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 77 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 78 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 79 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 80 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.004
Episode 81 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of

Episode 145 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 146 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 147 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 148 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 149 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.000
Episode 150 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.001
Episode 151 ## Reward 1 agent: 0.000 ##  Reward 2 : 0.090 ## Max: 0.090
Average Score of last 100 episodes: 0.002
Episode 152 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.002
Episode 153 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average 

Episode 217 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.017
Episode 218 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.018
Episode 219 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.018
Episode 220 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.019
Episode 221 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.019
Episode 222 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.019
Episode 223 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.019
Episode 224 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.020
Episode 225 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average

Episode 289 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.052
Episode 290 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.053
Episode 291 ## Reward 1 agent: 0.000 ##  Reward 2 : 0.090 ## Max: 0.090
Average Score of last 100 episodes: 0.054
Episode 292 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.054
Episode 293 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.054
Episode 294 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.054
Episode 295 ## Reward 1 agent: 0.000 ##  Reward 2 : 0.090 ## Max: 0.090
Average Score of last 100 episodes: 0.054
Episode 296 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.054
Episode 297 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average S

...

Episode 577 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.072
Episode 578 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.072
Episode 579 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.072
Episode 580 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.072
Episode 581 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.073
Episode 582 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.074
Episode 583 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.074
Episode 584 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.075
Episode 585 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average

Episode 649 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.074
Episode 650 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.074
Episode 651 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.074
Episode 652 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.074
Episode 653 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.074
Episode 654 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.073
Episode 655 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.072
Episode 656 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.072
Episode 657 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average

Episode 721 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.060
Episode 722 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.059
Episode 723 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.059
Episode 724 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.060
Episode 725 ## Reward 1 agent: 0.100 ##  Reward 2 : 0.090 ## Max: 0.100
Average Score of last 100 episodes: 0.061
Episode 726 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.061
Episode 727 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.061
Episode 728 ## Reward 1 agent: 0.000 ##  Reward 2 : -0.010 ## Max: 0.000
Average Score of last 100 episodes: 0.060
Episode 729 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average 

Episode 793 ## Reward 1 agent: 0.000 ##  Reward 2 : 0.090 ## Max: 0.090
Average Score of last 100 episodes: 0.058
Episode 794 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.057
Episode 795 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.058
Episode 796 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.059
Episode 797 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.000 ## Max: 0.000
Average Score of last 100 episodes: 0.059
Episode 798 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.059
Episode 799 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.059
Episode 800 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.060
Episode 801 ## Reward 1 agent: 0.000 ##  Reward 2 : 0.090 ## Max: 0.090
Average S

Episode 865 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.065
Episode 866 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.065
Episode 867 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.066
Episode 868 ## Reward 1 agent: -0.010 ##  Reward 2 : 0.100 ## Max: 0.100
Average Score of last 100 episodes: 0.067
Episode 869 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.068
Episode 870 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average Score of last 100 episodes: 0.069
Episode 871 ## Reward 1 agent: 0.100 ##  Reward 2 : 0.090 ## Max: 0.100
Average Score of last 100 episodes: 0.069
Episode 872 ## Reward 1 agent: 0.100 ##  Reward 2 : 0.090 ## Max: 0.100
Average Score of last 100 episodes: 0.070
Episode 873 ## Reward 1 agent: 0.100 ##  Reward 2 : -0.010 ## Max: 0.100
Average S